# Lesson 15: Deployment Part 2 
---
Intro: We will address final errors and that's it! We're almost finished making our website go live.

# Review
---

1. What is Heroku?
2. What does deployment mean?
1. What are environment variables?


# Concept 1: Final Error Checks
---


## Objective

1. Go to Git Bash. Let's ensure you are logged in to heroku with `heroku login` and you are in `tomas_project_desktop`.
2. Let's also double check the configurations using `heroku config`. You should see all the configurations we set earlier in this course.
3. If we don't have the `DISABLE_COLLECTSTATIC` configuration, then go to GitBash and type in this `heroku config:set DISABLE_COLLECTSTATIC=1`
* This should solve the POSIX error ([Stackoverflow](https://stackoverflow.com/questions/64327102/error-on-deploying-django-app-to-heroku-typeerror-argument-of-type-posixpat)). A POSIX provides os.path functionality. If it doesn't work, let's search the error.
4. `git add -A; git commit -m "Added POSIX Config"; git push heroku master; heroku open` - Pushes code to Heroku and opens it
5. If all works well, you should see an `OperationalError at /: no such table: blog_post`. This means we need to set up the database. Now databases are a different/complex topic so let's just keep it at a high-level. Django uses `db.sqlite` (See-Quill-Light) and now we are going to use a PostgreSQL (Post-Gress-Cue-Ell) database. As a high-level concept, just know they are two different databases but Postgres is better for production.

## Download Postgres
6. Let's download it using this [link](https://devcenter.heroku.com/articles/heroku-postgresql#set-up-postgres-on-windows). Follow the installation instructions. 

> Remember to update your PATH environment variable to add the bin directory of your Postgres installation. The directory will be similar to this: `C:\Program Files\PostgreSQL\<VERSION>\bin`. If you forget to update your PATH, commands like `heroku pg:psql` won’t work.

7. `heroku addons` - Type this in Git Bash. This checks if Heroku creates a database for our project already. If this checks out, you should see a database already set up.
8. `heroku addons:create heroku-postgresql:hobby-dev` - If a database was not created, then type this command. This sets up a database under the free tier of hobby-dev. Once that's created, check again using `heroku addons`.

## Django-Heroku Configurations 
Now that we have a database set up, let's create a super user. Recall we did this at the very beginning when creating our Django project. We will do this again to log in to our django admin site.
9. `pip install django-heroku` - This package will automatically configure our database url and handle all our static files.
10. Now let's import it. Return to Atom and in the project's `settings.py` file, type in `import django_heroku`. 
11. Now at the very bottom add this - `django_heroku.settings(locals())`
* This activates the django-heroku package to automatically configure your Django application to work on Heroku. 
12. Return to Git Bash and push your changes. `git add -A; git commit -m "Added django-heroku"; git push heroku master; heroku open`
13. We should see another error - `ProgrammingError at  - relation "blog_post" does not exist`. We're almost finished with these errors! 
14. The error just means we need to migrate our changes to the database. We don't need to make our migrations because the migrations are already pushed to Heroku. We just need to migrate them. Ensure you are in Git Bash at the root of the directory and type in - `heroku run python manage.py migrate`. `heroku run` runs a command on Heroku.
15. Let's create a superuser to log in to the admin page. We need to do this because we are starting off fresh with a new database. Type in - `heroku run python manage.py createsuperuser`. Enter a username,email, and now a more secure password that only you know. 
16. Lastly, open up Atom to change the debug variable - `
DEBUG = (os.environ.get("DEBUG_VALUE") == "True")`
* This gets the environment variable and returns True only if it is True. Otherwise, DEBUG will be False.
17. `git add -A; git commit -m "Set debug value"; git push heroku master` - Push our changes to Heroku
18. `heroku open` - This should open the website with no errors! The website is blank, but remember we made a whole new database on Heroku.


 


## DIY:
---

1. What are environment variables again?

# Concept 2: AWS for File Uploads (Optional)
---


## Objective
Recall that Django does not have a way to store uploaded static files. However, the fantastic WhiteNoise project (we'll see later) can integrate into your Django application, and was designed with exactly this purpose in mind. AWS (Amazon Web Services) is scalable, secure, and has good performance when it comes to static files.

> **Note**: AWS requires credit card information. AWS is free to use but if you go over the free-tier limit, you will be charged. The following steps are optional. If you choose not to use AWS, your website's pictures won't show. 

> Be cautious when uploading large images. This is where AWS charges 2 cents per Gigabyte. Be sure to caution your registered users about this.

> If you choose to use AWS and decide to delete it. Just know it's also a lengthy process and too complex for a beginner.
## What are they?
19. First off, let's make an account with AWS [here](https://aws.amazon.com/).
20. Once you are signed up, go to the AWS Management Console.
21. Search for S3 - Scalable storage in the cloud. S3 stands for simple storage service.
22. Next, we are going to create a bucket. A bucket is what holds our files. They have a unique ID different from any other ID in the world.
23. Click on Create Bucket. The bucket name is `django-blog-files` and leave the region at US West (Oregon) or the default region. Click Next, accept all defaults (leaving all unchecked/checked items as they are), click next until you see Create Bucket. Click on Create Bucket.
24. Click on you `django-blog-files` bucket. At the top menu, click on permissions, then CORS configuration. CORS enables us to interact with different domains or urls. Copy and paste the following code inside there and click save. This configuration code will request data from other domains. Notice the GET, POST, and PUT actions.


In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<CORSConfiguration xmlns="http://s3.amazonaws.com/doc/2006-03-01/">
   <CORSRule>
        <AllowedOrigin>*</AllowedOrigin>
        <AllowedMethod>GET</AllowedMethod>
        <AllowedMethod>POST</AllowedMethod>
        <AllowedMethod>PUT</AllowedMethod>
        <AllowedHeader>*</AllowedHeader>
    </CORSRule>
</CORSConfiguration>

25. Return to the AWS Management Console. Search for IAM which stands for Identity and Access Management that enables you to manage access to AWS services and resources securely. 
26. On the left sidebar, click on users. You should see the AWS_admin.
27. Let's create on add user and give the user name: `django_user`. Then click only on `programmatic access` as the access type. This will give us a key ID and access key which we will use in Django.
28. Click on `Next:Permissions`. Click on attach existing policies only and search for `AmazonS3FullAccess`. This will give the superuser (you) full access to S3 services. Click next all the way to create user.
29. Now you should see `django_user` with its access key id and secret access key. We will put these in our environment variables.
30. Go to Windows -> Control Panel -> System and Security -> System -> Advanced System settings -> Click on environment variables 
31. Create an evironment variable called `AWS_ACCESS_KEY_ID` and assign it with the access key id. Create another environment variable called `AWS_SECRET_ACCESS_KEY` and assign it with the secret access key. Lastly, create a third environment variable called `AWS_STORAGE_BUCKET_NAME` and assign it with `django-blog-files`.

## Modify Django
Now we need to make some changes in Django so it can work with our S3 services. Recall that S3 is just storage on the internet. 
32. `pip install boto3` - Return to GitBash. Ensure you are in the correct directory and type in the command. Boto3 is a Python package for the S3 service. We will interact with AWS S3 using Python.
33. `pip install django-storages` - django-storages is a collection of custom storage backends for Django. This enables Django to use S3.
34. Now return to Atom and let's edit the project's `settings.py` file. In `INSTALLED_APPS`, include `"storages"`.
35. At the very bottom include these variables -

```
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
AWS_STORAGE_BUCKET_NAME = os.environ.get('AWS_STORAGE_BUCKET_NAME')

AWS_S3_FILE_OVERWRITE = False
AWS_DEFAULT_ACL = None

DEFAULT_FILE_STORAGE = 'storages.backends.s3boto3.S3Boto3Storage'
```
> Note: these settings were taken from the [django-storages documentation](https://django-storages.readthedocs.io/en/latest/backends/amazon-S3.html). 
* Similar to past environment variables, we will retrieve the environment variables and place them into these AWS variables
* We set AWS_S3_FILE_OVERWRITE to False because we don't want a user's file to be overwritten. Example: Abel uses the default image. Bobby uses the default image. Even though they use the same image, we don't want the Bobby's image to be overwritten by Abel.
* By AWS standards, set the default ACL to None. ACL stands for access control list.
* Lastly, we set the default file storage to Amazon S3 services.
## Upload our photos
36. Next, go back to the AWS Management Console and search for S3. Click on it and click on our `django-blog-files` bucket.

> Note: Be cautious when uploading large images. This is where AWS charges 2 cents per Gigabyte. Be sure to caution your registered users about this.

37. Drag and drop or upload the `default.jpg` and `profile_pics` folder onto S3. These should be in the media folder. 
38. Once you place them to upload, it will ask for additional steps. Just click next until it says to upload.
39. That's it! Be sure to save the final changes and push to Heroku.

## Delete S3
If you chose to use S3 but decided that you don't want it anymore
It's best to follow directions [here](https://docs.aws.amazon.com/AmazonS3/latest/user-guide/delete-bucket.html) or [here](https://docs.aws.amazon.com/AmazonS3/latest/dev/delete-or-empty-bucket.html) as well as doing your own research.

# Summary:
---


1. We set up a database to work on Heroku.
2. What is Postgres?

# Homework:
---



1. Study for the final exam by going through the DIY questions. If you don't know the answer to the DIY questions, most of them are in the lessons or in the code. If you still are unsure, feel free to send an email!